While testing the unstable sandbox environment on 19/8/25, encountered an issue with using the fractional cover package with the new datacube 1.9

FC repo for context: https://github.com/GeoscienceAustralia/fc

If you run this script with datacube==1.9, the code will throw an error.

If you un-comment the lines that uninstall datacube and re-install datacube 1.8, then the script will execute without error.

In [1]:
%xmode verbose


Exception reporting mode: Verbose


In [2]:
%pip install fractional_cover --find-links="https://packages.dea.ga.gov.au/fc" -q


Note: you may need to restart the kernel to use updated packages.


In [ ]:
# %pip uninstall datacube -y
# %pip install datacube==1.8.0 -q


In [ ]:
import os
import yaml
import datacube
import numpy as np
import pandas as pd
import xarray as xr
import geopandas as gpd

from fc.fractional_cover import (
    fractional_cover,
)  # import the FC package after installing it above

from odc.geo.xr import assign_crs
from odc.geo.geom import Geometry
from odc.algo import keep_good_only
from odc.algo._percentile import xr_quantile_bands
from datacube.model import Measurement
from datacube.utils import unsqueeze_dataset, masking

import sys

sys.path.insert(1, "/home/jovyan/dev/Tools/")
from dea_tools.datahandling import load_ard
from dea_tools.dask import create_local_dask_cluster


In [4]:
!pip list | grep datacube


datacube                          1.9.7
datacube_ows                      1.9.4


There is a yaml file in this directory that contains the additional information needed to run the fractional cover function alter on. 

In [ ]:
# Read YAML file
with open("/home/jovyan/git/fc-sub-annual/notebooks/variables.yaml", "r") as f:
    config = yaml.safe_load(f)

# Extract variables
MEASUREMENTS = config["measurements"]
LANDSAT_SENSOR_REGRESSION_COEFFICIENTS = config[
    "landsat_sensor_regression_coefficients"
]
SENTINEL_SENSOR_REGRESSION_COEFFICIENTS = config[
    "sentinel_sensor_regression_coefficients"
]
LANDSAT_BAND_MAPPING = config["landsat_band_mapping"]
SENTINEL_BAND_MAPPING = config["sentinel_band_mapping"]

# Convert measurements to Measurement objects
MEASUREMENTS_OBJ = [Measurement(**m) for m in MEASUREMENTS]


In [ ]:
dc = datacube.Datacube(app="fc_odc_unstable_img_bug")


In [ ]:
lat = -35.5200
lon = 144.0000
buffer = 0.01

lat_range = (lat - buffer, lat + buffer)
lon_range = (lon - buffer, lon + buffer)


In [ ]:
query = {
    "x": lon_range,
    "y": lat_range,
    "time": ("2024-01-01", "2024-01-31"),
    "resolution": (-30, 30),
    "group_by": "solar_day",
    "output_crs": "EPSG:3577",
}


In [ ]:
ls_ds = load_ard(
    dc=dc,
    products=["ga_ls8c_ard_3", "ga_ls9c_ard_3"],
    measurements=[
        "nbart_green",
        "nbart_red",
        "nbart_nir",
        "nbart_swir_1",
        "nbart_swir_2",
    ],
    cloud_mask="fmask",
    mask_pixel_quality=True,
    fmask_categories=["valid", "snow", "water"],
    mask_filters=[("dilation", 5)],
    mask_contiguity=True,
    skip_broken_datasets=True,
    verbose=True,
    **query
)


Finding datasets
    ga_ls8c_ard_3
    ga_ls9c_ard_3
Applying morphological filters to pixel quality mask: [('dilation', 5)]
Applying fmask pixel quality/cloud mask
Applying contiguity mask (oa_nbart_contiguity)
Loading 7 time steps


/home/jovyan/dev/Tools/dea_tools/datahandling.py:519: UserWarning: As of `dea_tools` v0.3.0, pixel quality masks are inverted before being passed to `mask_filters` (i.e. so that good quality/clear pixels are False and poor quality pixels/clouds are True). This means that 'dilation' will now expand cloudy pixels, rather than shrink them as in previous versions.
  warnings.warn(
/env/lib/python3.10/site-packages/rasterio/warp.py:387: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dest = _reproject(


In [ ]:
# band renaming to make sure the bands are named correctly for the fractional cover function
for mapping in LANDSAT_BAND_MAPPING:
    if mapping["rename"]:
        ls_rename_dict = mapping["rename"]
        break

ls_renamed = ls_ds.rename(ls_rename_dict)


In [ ]:
test_da = ls_renamed.isel(time=0)
test_da


<xarray.Dataset> Size: 111kB
Dimensions:      (y: 81, x: 68)
Coordinates:
    time         datetime64[ns] 8B 2024-01-01T00:09:22.123303
  * y            (y) float64 648B -3.93e+06 -3.93e+06 ... -3.932e+06 -3.932e+06
  * x            (x) float64 544B 1.084e+06 1.084e+06 ... 1.086e+06 1.086e+06
    spatial_ref  int32 4B 3577
Data variables:
    green        (y, x) float32 22kB nan nan nan nan ... 491.0 506.0 513.0 510.0
    red          (y, x) float32 22kB nan nan nan nan ... 489.0 512.0 493.0 500.0
    nir          (y, x) float32 22kB nan nan nan ... 2.331e+03 2.247e+03
    swir1        (y, x) float32 22kB nan nan nan ... 1.338e+03 1.398e+03
    swir2        (y, x) float32 22kB nan nan nan nan ... 757.0 783.0 749.0 790.0
Attributes:
    crs:           EPSG:3577
    grid_mapping:  spatial_ref

In [ ]:
ls_fc = fractional_cover(
    test_da, MEASUREMENTS_OBJ, LANDSAT_SENSOR_REGRESSION_COEFFICIENTS
)


StopIteration: 